In [1]:
"""%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
"""
import sys
import json
from ruamel.yaml import YAML
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import gc

import lossmaps as lm

from scipy.constants import c as clight

from cpymad.madx import Madx

import xpart as xp
import xtrack as xt
import xobjects as xo
import xdyna as xd
import xcoll as xc
import xfields as xf
import xdeps as xdeps
# import xboinc as xb

import subprocess
import os

print('xcoll version: ', xc.__version__)
print('xtrack version: ', xt.__version__)
print('xpart version: ', xp.__version__)
print('xobjects version: ', xo.__version__)
print('xfields version: ', xf.__version__)
print('xdeps version: ', xd.__version__)

No pytransport functionality
No pytransport functionality
xcoll version:  0.3.4
xtrack version:  0.54.1
xpart version:  0.17.0
xobjects version:  0.4.1
xfields version:  0.16.0
xdeps version:  0.1.1


In [4]:
# context = xo.ContextCpu(omp_num_threads=26)
context = xo.ContextCpu()
context.allow_prebuilt_kernels = False
context.allow_prebuilt_kernels

False

### Crystal Setup

In [5]:
#branch_name = 'main'
#branch_name = 'release_v0.4.0'
#branch_name = 'NoMoreManager'
branch_name = 'KayStatus'
#branch_name = 'ForKayAsSixtrack'
#branch_name = 'ForKayBetterEtaWithTransition'
#branch_name = 'ForKayBetterEtaWithoutTransition'
#branch_name = 'sixtrack'

In [41]:
if branch_name in ['KayStatus', 'ForKayAsSixtrack', 'ForKayBetterEtaWithTransition', 'ForKayBetterEtaWithoutTransition']:
    coll = xc.EverestCrystal(length=0.002, material=xc.materials.SiliconCrystal, bending_angle=144e-6,
                            xdim=0.002, ydim=0.05, side='+', lattice='strip', jaw=0.001)
elif branch_name == 'main':
    coll = xc.EverestCrystal(length=0.002, material=xc.materials.SiliconCrystal, bending_angle=144e-6,
                            xdim=0.002, ydim=0.05, side='+', lattice='strip', jaw=0.001)
elif branch_name in ['NoMoreManager', 'release_v0.4.0']:
    coll = xc.EverestCrystal(length=0.002, material=xc.materials.SiliconCrystal, bending_angle=144e-6,
                            width=0.002, height=0.05, side='+', lattice='strip', jaw=0.001)
elif branch_name == 'sixtrack':
    class collimator():
        def __init__(self, bending_angle):
            self.bending_angle = bending_angle
    coll = collimator(bending_angle=144e-6)
    
xp_crit_comp = 9.130328408646464e-06

In [7]:
EOS_PATH  = Path('/eos/home-c/cmaccani/xsuite_sim/two_cryst_sim/Condor/xcoll_vers_test')

### Tracking

In [ ]:
tot_part = 0

for i in range(300):
    
    print('\nIteration: ', i, ",    range: ", (-50+i), ' , ',(-51+i), 'urad')
    num_part = int(3.33334e6)
    x_init   = np.random.normal(loc=1.5e-3, scale=75.e-6, size=num_part)
    px_init  = np.random.uniform(low=(-50+i)*1.e-6, high=(-51+i)*1.e-6, size=num_part)
    y_init   = np.random.normal(loc=0., scale=1e-6, size=num_part)
    py_init  = np.random.normal(loc=0., scale=1.e-8, size=num_part)
    part = xp.Particles(x=x_init, px=px_init, y=y_init, py=py_init, delta=0, p0c=4e11)

    del x_init, px_init, y_init, py_init
    gc.collect()

    part_init_px = part.px.copy().astype('float32')


    %time coll.track(part)
    # Sort particles again
    part.sort(interleave_lost_particles=True)

    alive = (part.state > 0 )
    df = pd.DataFrame({'px_in': part_init_px[alive], 'px_diff': part.px[alive].astype('float32') - part_init_px[alive]})
    mask =  (df.px_diff < 200.e-6) & ( df.px_diff > -75.e-6)

    del part, part_init_px
    gc.collect()

    # if i ==0:
    df[mask].to_hdf(Path( EOS_PATH / 'slices', f'branch_{branch_name}_{-50+i}urad.h5'), key='px', format='table', mode='a',
                complevel=9, complib='blosc')
    # else:
    #     store = pd.HDFStore(Path(Path.cwd(), f'branch_{branch_name}_{-50+i}urad.h5'))
    #     store.append('px', df)
    #     store.close()
    
    tot_part += len(df)
    print('N part: ' ,len(df))
    
    del df
    gc.collect()


print('Total particles: ', tot_part)


In [9]:
print('Total particles:  ', tot_part)


Total particles:   996094120


### Loading


In [24]:
file_list = os.listdir(Path(EOS_PATH /'slices').absolute().as_posix())
file_list = [f for f in file_list if branch_name in f]
files = {}
for file in file_list:
    files[int(file.split('_')[-1].split('u')[0])-1] = file

In [26]:
if branch_name == 'sixtrack':
    files = {k + 1: v for k, v in files.items()}

In [28]:
files

{-10: 'branch_sixtrack_-10urad.h5',
 -11: 'branch_sixtrack_-11urad.h5',
 -12: 'branch_sixtrack_-12urad.h5',
 -13: 'branch_sixtrack_-13urad.h5',
 -14: 'branch_sixtrack_-14urad.h5',
 -15: 'branch_sixtrack_-15urad.h5',
 -16: 'branch_sixtrack_-16urad.h5',
 -17: 'branch_sixtrack_-17urad.h5',
 -18: 'branch_sixtrack_-18urad.h5',
 -19: 'branch_sixtrack_-19urad.h5',
 -1: 'branch_sixtrack_-1urad.h5',
 -20: 'branch_sixtrack_-20urad.h5',
 -21: 'branch_sixtrack_-21urad.h5',
 -22: 'branch_sixtrack_-22urad.h5',
 -23: 'branch_sixtrack_-23urad.h5',
 -24: 'branch_sixtrack_-24urad.h5',
 -25: 'branch_sixtrack_-25urad.h5',
 -26: 'branch_sixtrack_-26urad.h5',
 -27: 'branch_sixtrack_-27urad.h5',
 -28: 'branch_sixtrack_-28urad.h5',
 -29: 'branch_sixtrack_-29urad.h5',
 -2: 'branch_sixtrack_-2urad.h5',
 -30: 'branch_sixtrack_-30urad.h5',
 -31: 'branch_sixtrack_-31urad.h5',
 -32: 'branch_sixtrack_-32urad.h5',
 -33: 'branch_sixtrack_-33urad.h5',
 -34: 'branch_sixtrack_-34urad.h5',
 -35: 'branch_sixtrack_-35urad.h

In [62]:
def load(low_lim, high_lim = None, files=files, perc = 1):
    

    with pd.HDFStore(Path(EOS_PATH, 'slices', files[low_lim]), 'r') as store:
        total_rows = store.get_storer('px').nrows
    store.close()
    rows_to_read = int(total_rows * perc)
    df = pd.read_hdf(Path(EOS_PATH, 'slices', files[low_lim]), key='px', start=0, stop=rows_to_read)


    if high_lim is not None:
        for i in range(low_lim+1, high_lim):
            with pd.HDFStore(Path(EOS_PATH, 'slices', files[i]), 'r') as store:
                total_rows = store.get_storer('px').nrows
            store.close()
            rows_to_read = int(total_rows * perc)
            df = pd.concat([df, pd.read_hdf(Path(EOS_PATH, 'slices', files[i]), key='px', start=0, stop=rows_to_read)])

    if branch_name == 'release_v0.4.0':
        df = df.rename(columns={'part_init_px': 'px_in', 'part_Diff_px': 'px_diff'})

    if branch_name == 'sixtrack':
        df = df[(df.px_in > -50e-6) & (df.px_in < 206e-6)]
        if low_lim < 0:
            low_lim = low_lim -1

    if high_lim is not None:
        df = df[(df.px_in > low_lim*1e-6) & (df.px_in < high_lim*1e-6)]
    else:
        df = df[(df.px_in > low_lim*1e-6) & (df.px_in < (low_lim+1)*1e-6)]
        

    return df

### Plotting